In [1]:
import pickle
import utils
from process_data import * 
import numpy as np
logs_path="./logs_5"

with open ('art_filt.txt', 'rb') as fp:
    articulos_filtrados = pickle.load(fp)
with open ('art_filt_labels', 'rb') as fp:
    articulos_filtrados_labels = pickle.load(fp)

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/cselmo/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/cselmo/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
[nltk_data] Downloading package stopwords to /home/cselmo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Extracting features from articles
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(max_df=0.8,min_df=10)
count_vect.fit(articulos_filtrados) #Aprende el vocabulario y le asigna un código a cada palabra

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=10,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [3]:
count_vect.vocabulary_ #Estos son los índices de cada una de las palabras

{'re': 6547,
 'cub': 1884,
 'behind': 735,
 'marlin': 4872,
 'how': 3754,
 'univers': 8503,
 'california': 1126,
 'berkeley': 771,
 'gajarski': 3183,
 'write': 9028,
 'morgan': 5220,
 'era': 2661,
 'run': 6964,
 'higher': 3650,
 'last': 4507,
 'year': 9088,
 'idiot': 3837,
 'pitch': 6082,
 'much': 5274,
 'wo': 8973,
 'good': 3342,
 'think': 8120,
 'stud': 7766,
 'pitcher': 6083,
 'thi': 8111,
 'season': 7123,
 'far': 2856,
 'help': 3604,
 'lead': 4539,
 'top': 8229,
 'even': 2727,
 'better': 786,
 'the': 8082,
 'rotat': 6927,
 'atlanta': 522,
 'brave': 971,
 'we': 8834,
 'know': 4428,
 'earli': 2464,
 'fan': 2850,
 'learn': 4547,
 'enjoy': 2619,
 'short': 7283,
 'triumph': 8338,
 'still': 7701,
 'gregori': 3406,
 'nelson': 5407,
 'thank': 8078,
 'appl': 379,
 'free': 3112,
 'ethernet': 2707,
 'rutger': 6976,
 'new': 5430,
 'brunswick': 1021,
 'well': 8868,
 'get': 3266,
 'centri': 1265,
 'yesterday': 9098,
 'it': 4169,
 'take': 7947,
 'two': 8398,
 'week': 8854,
 'place': 6095,
 'order

In [4]:
vocabulary=count_vect.get_feature_names() #Estos nombres de las palabras seleccionadas para el vocabulario, ordenadas por orden alfabético
print(vocabulary)

['aa', 'aaa', 'aaron', 'ab', 'abandon', 'abbrevi', 'abc', 'aberystwyth', 'abid', 'abil', 'abl', 'abnorm', 'aboard', 'abolish', 'abomin', 'abort', 'abound', 'about', 'abov', 'abpsoft', 'abraham', 'abridg', 'abroad', 'absenc', 'absent', 'absolut', 'absorb', 'abstract', 'absurd', 'abund', 'abus', 'ac', 'academ', 'academi', 'academia', 'acc', 'acceler', 'acceller', 'accept', 'access', 'accessori', 'accid', 'accident', 'accommod', 'accomod', 'accompani', 'accompli', 'accomplish', 'accord', 'accordingli', 'account', 'accumul', 'accur', 'accuraci', 'accus', 'ace', 'ach', 'achiev', 'achkar', 'acid', 'ack', 'acker', 'acknowledg', 'aclu', 'acm', 'acquaint', 'acquir', 'acquisit', 'acronym', 'across', 'act', 'action', 'activ', 'activist', 'actual', 'acut', 'ad', 'adam', 'adapt', 'adaptec', 'adaptor', 'adb', 'adc', 'adcom', 'add', 'addict', 'addit', 'addr', 'address', 'adequ', 'adher', 'adirondack', 'adjac', 'adject', 'adjust', 'adl', 'admin', 'administ', 'administr', 'admir', 'admiss', 'admit', 'a

In [5]:
X_train_data=count_vect.fit_transform(articulos_filtrados) #Aprende el vocabulario y le asigna un código a cada palabra

In [6]:
print(X_train_data.shape) #Para cada documento hay un vector de ocurrencias
X_train_data=X_train_data.toarray()

(11314, 9140)


In [7]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)
#labels=tf.one_hot(articulos_filtrados_labels, 20)
labels = np.zeros((X_train_data.shape[0], 20))
labels[np.arange(X_train_data.shape[0]), articulos_filtrados_labels] = 1

In [ ]:
import tensorflow as tf

# Parameters
learning_rate = 0.01
training_epochs = 15
batch_size = 10
display_step = 1

# Network Parameters
n_input =  X_train_data.shape[1] # Twenty groups dataset
n_classes = 20 # Twenty groups total classes

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias

weights= tf.Variable(tf.random_normal([n_input, n_classes]))
bias= tf.Variable(tf.random_normal([n_classes]))


logits = tf.matmul(X, weights) + bias

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    total_batch = int(X_train_data.shape[0]/batch_size)
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,X_train_data,labels)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: X_train_data, Y: labels}))

In [8]:
import tensorflow as tf
import shutil
from tensorboard import summary as summary_lib
shutil.rmtree(logs_path)
# Parameters
learning_rate = 0.01
training_epochs = 50
batch_size = 256
display_step = 1
hidden_units=50

# Network Parameters
n_input =  X_train_data.shape[1] # Vocab size 
n_classes = 20 # Twenty news groups # classes


with tf.name_scope("inputs"):
    # tf Graph input
    X = tf.placeholder("float", [None, n_input],name="X")
with tf.name_scope("labels"):
    Y = tf.placeholder("float", [None, n_classes],name="Y")

# Construct model
with tf.name_scope('Capa1'):
    # Model
    weights1= tf.Variable(tf.random_normal([n_input, hidden_units]),name="weights1")
    bias1= tf.Variable(tf.random_normal([hidden_units]),name="bias1")
    act1= tf.nn.sigmoid(tf.matmul(X,weights1)+bias1, name="activacion_1")

with tf.name_scope('Capa2'):
    # Model
    weights2= tf.Variable(tf.random_normal([hidden_units, n_classes]),name="weights2")
    bias2= tf.Variable(tf.random_normal([n_classes]),name="bias2")
    logits= tf.matmul(act1,weights2)+bias2

with tf.name_scope('Loss'):
# Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=Y),name="costo")
with tf.name_scope('BGD'):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate,name="optimizador")
    train_op = optimizer.minimize(loss_op)
with tf.name_scope('Accuracy'):
    # Accuracy
    #pred = tf.nn.softmax(logits) # Softmax
    acc_op = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    acc_op = tf.reduce_mean(tf.cast(acc_op, tf.float32),name="acc_red_mean")
    
# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", loss_op)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc_op)
# Merge all summaries into a single op
tf.summary.histogram('histogram', weights1)
merged_summary_op = tf.summary.merge_all()


with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=sess.graph)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(X_train_data.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = next_batch(batch_size,X_train_data,labels)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c= sess.run([train_op, loss_op], feed_dict={Y: batch_y,
                                                            X: batch_x})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            batch_x, batch_y = next_batch(batch_size,X_train_data,labels)
            #run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
            #run_metadata = tf.RunMetadata()
            summary, _,_ = sess.run([merged_summary_op,loss_op,acc_op],
                                  feed_dict={X: X_train_data, Y: labels})#,
                                  #options=run_options,
                                  #run_metadata=run_metadata)
            summary_writer.add_summary(summary, epoch)
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: X_train_data, Y: labels})) 

/home/cselmo/anaconda3/envs/OpLaDyn/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Epoch: 0001 cost=5.179628849
Epoch: 0002 cost=2.367291483
Epoch: 0003 cost=1.365297599
Epoch: 0004 cost=0.869607702
Epoch: 0005 cost=0.642310436
Epoch: 0006 cost=0.448158211
Epoch: 0007 cost=0.360061840
Epoch: 0008 cost=0.282971728
Epoch: 0009 cost=0.217510563
Epoch: 0010 cost=0.167690058
Epoch: 0011 cost=0.144490402
Epoch: 0012 cost=0.119196838
Epoch: 0013 cost=0.107108994
Epoch: 0014 cost=0.092991551
Epoch: 0015 cost=0.077079192
Epoch: 0016 cost=0.067969509
Epoch: 0017 cost=0.060078999
Epoch: 0018 cost=0.055311024
Epoch: 0019 cost=0.052702332
Epoch: 0020 cost=0.045579092
Epoch: 0021 cost=0.041233425
Epoch: 0022 cost=0.034736988
Epoch: 0023 cost=0.035429245
Epoch: 0024 cost=0.032668386
Epoch: 0025 cost=0.035846888
Epoch: 0026 cost=0.025663181
Epoch: 0027 cost=0.024628141
Epoch: 0028 cost=0.026033282
Epoch: 0029 cost=0.023205513
Epoch: 0030 cost=0.022687738
Epoch: 0031 cost=0.020631501
Epoch: 0032 cost=0.015122659
Epoch: 0033 cost=0.017527347
Epoch: 0034 cost=0.017824071
Epoch: 0035 co